In [1]:
import sys
sys.path.append('../')

In [2]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import model.metric as module_metric

from data_loader.dataloader import get_dataloader 
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from model.loss import *
from train import *
from pathlib import Path

In [3]:
SEED = 42
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 4
num_epoch = 100
refined = 'unrefined'

train_dir = f'./dataset/{refined}/crushed/train/'
val_dir = f'./dataset/refined/crushed/val/'

In [5]:
transform_train = A.Compose([
    A.HorizontalFlip(),
    A.Rotate((-10, 10), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
    A.Resize(224, 224),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

In [6]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size)

In [7]:
model = smp.Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)
model = model.to(device)

In [8]:
criterion = IoULoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5* 1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=50, T_mult=2, eta_min=1e-7)
metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

In [9]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch

train_config['Loss fn'] = criterion.__class__.__name__
train_config['Optimizer'] = optimizer.__class__.__name__
train_config['LR Scheduler'] = scheduler.__class__.__name__
train_config['Metric'] = {str(idx+1) : metric for idx, metric in enumerate([metrics[i].__name__ for i in range(len(metrics))])}


In [10]:
wandb.init(project='Segmentation', name=f"U-Net_{model.__dict__['name'].split('-')[1:][0].capitalize()}_{refined}", config=train_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: imlim. Use `wandb login --relogin` to force relogin


In [11]:
save_dir = f"./saved/U-Net/{model.__dict__['name'].split('-')[1:][0].capitalize()}_Ver0/{refined}"

trainer = Trainer(model, criterion, metrics, optimizer, device, num_epoch, save_dir,
                                  data_loader=train_dataloader, valid_data_loader=val_dataloader,
                                  lr_scheduler=scheduler)

In [12]:
trainer.train()


Epoch : 0 | Train Loss : 0.92417 | Train P.A : 84.41% | Train IOU : 0.14763 | Val Loss : 0.99452 | Val P.A : 97.74% | Val IOU : 0.00234 | Training Time : 36.85sec

Epoch : 1 | Train Loss : 0.84810 | Train P.A : 96.52% | Train IOU : 0.30111 | Val Loss : 0.99125 | Val P.A : 97.62% | Val IOU : 0.00844 | Training Time : 34.76sec

Epoch : 2 | Train Loss : 0.82095 | Train P.A : 97.10% | Train IOU : 0.33754 | Val Loss : 0.99831 | Val P.A : 97.69% | Val IOU : 0.00194 | Training Time : 33.87sec

Epoch : 3 | Train Loss : 0.81121 | Train P.A : 97.24% | Train IOU : 0.35035 | Val Loss : 0.99946 | Val P.A : 97.74% | Val IOU : 0.00000 | Training Time : 34.82sec

Epoch : 4 | Train Loss : 0.80410 | Train P.A : 97.44% | Train IOU : 0.37167 | Val Loss : 0.97010 | Val P.A : 97.28% | Val IOU : 0.02038 | Training Time : 33.58sec

Epoch : 5 | Train Loss : 0.79641 | Train P.A : 97.51% | Train IOU : 0.35830 | Val Loss : 0.97683 | Val P.A : 97.60% | Val IOU : 0.02456 | Training Time : 33.47sec

Epoch : 6 | Tra

Train IOU,▁▅▅▆▆▆▆▆▇▇▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇█▇██▇▇██▇▇▇▇▇██
Train Loss,█▅▄▄▃▄▃▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁▂▁▂▁▂
Train P.A,▁▇██████████████████████████████████████
Val IOU,▁▂▁▅▆▁▅▃▂▂▃▇▇▂▂▁▃█▅▂▂▂▆▃▄▄▃▅▃▂▄▂▁▄▃▃▄▂▁▄
Val Loss,▇▇█▄▅█▄▇▆▆▆▄▂▇▇█▆▁▄▇▇▇▂▇▆▆▆▆▆▇▄▇▇▄▅▆▆▇█▆
Val P.A,█▇█▅▇▇▇███▇▅▁▇██▆▄▆██▆▆▇▅▆▆▆▆▇▇▆▇▄▅▆▆▇▇▆
Train IOU,0.44684
Train Loss,0.75828
Train P.A,97.80989
Val IOU,0.01576
Val Loss,0.98589
